In [ ]:
# load the data that was previously saved
def load_data(file_name):
    try:
        with open(file_name, "r") as f:
            lines = f.readlines()
            list_lors = []
            list_slices = []
            list_nlors = []
            list_nvoxels = []
            list_forward_timing = []
            list_backward_timing_triton = []
            list_mlem_timing_triton = []
            for line in lines:
                parts = line.split()
                if len(parts) == 7:
                    lors, slices, number_of_lors, number_of_voxels, forward_timing, backward_timing, mlem_timing = parts
                    print(f"Loaded data: LOR: {lors}, Slice: {slices}, Number of LORs: {number_of_lors}, Number of Voxels: {number_of_voxels}, Forward Timing: {forward_timing}, Backward Timing: {backward_timing}, MLEM Timing: {mlem_timing}")
                    list_lors.append(int(lors))
                    list_slices.append(int(slices))
                    list_nlors.append(int(number_of_lors))
                    list_nvoxels.append(int(number_of_voxels))
                    list_forward_timing.append(float(forward_timing))
                    list_backward_timing_triton.append(float(backward_timing))
                    list_mlem_timing_triton.append(float(mlem_timing))
            print("Data loaded successfully.")
        return list_lors, list_slices, list_nlors, list_nvoxels, list_forward_timing, list_backward_timing_triton, list_mlem_timing_triton
    except FileNotFoundError:
        print(f"File {file_name} not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

lors_triton, slices_triton, nlors_triton, nvoxels_triton, forward_timing_triton, backward_timing_triton, mlem_timing_triton = load_data("TIMING_triton_joseph_proj.txt")
lors_pp, slices_pp, nlors_pp, nvoxels_pp, forward_timing_pp, backward_timing_pp, mlem_timing_pp = load_data("TIMING_parallel_proj.txt")

In [ ]:
from matplotlib import pyplot as plt

nlors_nvoxels_pp = [nlors_pp[i] * nvoxels_pp[i] for i in range(len(nlors_pp))]
nlors_nvoxels_triton = [nlors_triton[i] * nvoxels_triton[i] for i in range(len(nlors_triton))]

# order pp and triton data by number of LORs * number of voxels
nlors_nvoxels_pp, forward_timing_pp, backward_timing_pp, mlem_timing_pp = zip(*sorted(zip(nlors_nvoxels_pp, forward_timing_pp, backward_timing_pp, mlem_timing_pp)))
nlors_nvoxels_triton, forward_timing_triton, backward_timing_triton, mlem_timing_triton = zip(*sorted(zip(nlors_nvoxels_triton, forward_timing_triton, backward_timing_triton, mlem_timing_triton)))

def percentage_difference_between_first_second_lists(list1, list2):
    return [(list2[i] - list1[i]) / list1[i] * 100 for i in range(len(list1))]

diff_per_for = percentage_difference_between_first_second_lists(forward_timing_pp, forward_timing_triton)
diff_per_back = percentage_difference_between_first_second_lists(backward_timing_pp, backward_timing_triton)
diff_per_mlem = percentage_difference_between_first_second_lists(mlem_timing_pp, mlem_timing_triton)
plt.figure(figsize=(8, 5))
plt.plot(nlors_nvoxels_pp, diff_per_for, 'ro', label='Forward')
plt.plot(nlors_nvoxels_triton, diff_per_back, 'bo', label='Backward')
plt.plot(nlors_nvoxels_triton, diff_per_mlem, 'go', label='MLEM step')
plt.axhline(0, color='black', linestyle='--', label='No Difference')
plt.xlabel('Number of LORs by Number of Voxels')
plt.ylabel('Timing Percentage Difference versus Parallel Proj (%)')
plt.legend()
plt.grid()
plt.xscale('log')
plt.savefig('timing_percentage_difference.png')